In [21]:
import pandas as pd
import re
import string
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import pandas as pd
import numpy as np
import scipy as sp
import sklearn
import sys
import nltk
from nltk.corpus import stopwords
from gensim.models import ldamodel
import gensim.corpora
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import NMF
from sklearn.preprocessing import normalize
import pickle
from gensim.models import LdaModel,LsiModel
from gensim.corpora import Dictionary
# import pyLDAvis.gensim
import warnings
from itertools import chain
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()
import numpy as np
import pandas as pd
import string
import re
from tqdm._tqdm_notebook import tqdm_notebook,tnrange,tqdm
from collections import Counter,OrderedDict
from gensim import models,corpora
# from gensim.summarization import summarize,keywords
import warnings
# import pyLDAvis.gensim
import matplotlib.pyplot as plt
import seaborn as sns
# import pyLDAvis.gensim
import gensim.models.phrases as gen
from gensim.models.coherencemodel import CoherenceModel

In [2]:
import gdown

In [3]:
id = 'https://drive.google.com/uc?id=1b3rmCSIoh6VhD4HKWjI4HOW-cSwcwbeC'
output = 'arxiv-dataset.zip'

gdown.download(id, output, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1b3rmCSIoh6VhD4HKWjI4HOW-cSwcwbeC
From (redirected): https://drive.google.com/uc?id=1b3rmCSIoh6VhD4HKWjI4HOW-cSwcwbeC&confirm=t&uuid=f9afd6e2-62b2-4a37-b449-960becbca56f
To: /content/arxiv-dataset.zip
100%|██████████| 3.62G/3.62G [00:33<00:00, 110MB/s]


'arxiv-dataset.zip'

In [4]:
!unzip arxiv-dataset.zip

Archive:  arxiv-dataset.zip
   creating: arxiv-dataset/
  inflating: __MACOSX/._arxiv-dataset  
  inflating: arxiv-dataset/train.txt  
  inflating: __MACOSX/arxiv-dataset/._train.txt  
  inflating: arxiv-dataset/vocab     
  inflating: __MACOSX/arxiv-dataset/._vocab  
  inflating: arxiv-dataset/test.txt  
  inflating: __MACOSX/arxiv-dataset/._test.txt  
  inflating: arxiv-dataset/val.txt   
  inflating: __MACOSX/arxiv-dataset/._val.txt  


In [5]:
train_file_path = "/content/arxiv-dataset/train.txt"
test_file_path = "/content/arxiv-dataset/test.txt"
val_file_path = "/content/arxiv-dataset/val.txt"
vocab_file_path = "/content/arxiv-dataset/vocab"

In [6]:
import json
import pandas as pd

In [7]:
def read_json_instances_up_to_limit(file_path, limit=25000):
    data = []
    with open(file_path, 'r') as file:
        instance_count = 0
        while instance_count < limit:
            line = file.readline()
            if not line:
                break  # End of file
            instance = json.loads(line)
            data.append(instance)
            instance_count += 1
    return data

# Read exactly 25,000 training instances


In [8]:
train_data = read_json_instances_up_to_limit(train_file_path, limit=15000)

In [9]:
train_df = pd.DataFrame(train_data)

In [26]:
test_data = read_json_instances_up_to_limit(test_file_path,limit=10000)

In [27]:
test_df = pd.DataFrame(test_data)

In [10]:
import pandas as pd

# Assuming train_df is your DataFrame
for index, row in train_df.iterrows():
    concatenated_article_text = ' '.join(row['article_text']) if isinstance(row['article_text'], list) else row['article_text']
    concatenated_abstract_text = ' '.join(row['abstract_text']) if isinstance(row['abstract_text'], list) else row['abstract_text']
    train_df.at[index, 'article_text'] = concatenated_article_text
    train_df.at[index, 'abstract_text'] = concatenated_abstract_text


In [28]:
import pandas as pd

# Assuming train_df is your DataFrame
for index, row in test_df.iterrows():
    concatenated_article_text = ' '.join(row['article_text']) if isinstance(row['article_text'], list) else row['article_text']
    concatenated_abstract_text = ' '.join(row['abstract_text']) if isinstance(row['abstract_text'], list) else row['abstract_text']
    test_df.at[index, 'article_text'] = concatenated_article_text
    test_df.at[index, 'abstract_text'] = concatenated_abstract_text


In [11]:
data=train_df
len(data)

15000

In [12]:
docs = [x for x in data['article_text']]

In [13]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [14]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['xcite', 'fig', 'eq', 'xmath','et','al','following','proof','let','theorm','mu','delta','gamma','lambda','omega','pi','sigma','data','research','result','beta'])

In [15]:
# A function to prepare the text for topic modelling
def words(text):
    regex = re.compile('[' + re.escape(string.punctuation) + '0-9\\r\\t\\n]')
    text = regex.sub(" ", text.lower())
    words = text.split()
    words = [re.sub('\S*@\S*\s?', '', sent) for sent in words]
    words = [re.sub('\s+', ' ', sent) for sent in words]
    words = [re.sub("\'", "", sent) for sent in words]
    words = [w for w in words if not len(w) < 2]
    words = [w for w in words if w not in stop_words]
    words = [lmtzr.lemmatize(w) for w in words]
    words = [w for w in words if not w.startswith('xmath')]  # Filter out words starting with 'xmath'

    return words

In [16]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [17]:
docs = [words(x) for x in data['article_text']]

In [18]:
dictionary = Dictionary(docs)

In [19]:
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [22]:
num_topics = 10  # Number of topics/components
lsa_model = LsiModel(corpus, id2word=dictionary, num_topics=num_topics)

# Step 4: Interpret the Results
for topic_id, topic in lsa_model.print_topics(num_topics):
    print("Topic {}: {}".format(topic_id, topic))

Topic 0: 0.191*"one" + 0.165*"model" + 0.158*"two" + 0.152*"function" + 0.149*"state" + 0.141*"energy" + 0.137*"right" + 0.135*"time" + 0.134*"case" + 0.133*"field"
Topic 1: 0.369*"right" + 0.348*"left" + 0.336*"frac" + 0.253*"hline" + 0.242*"tabular" + 0.197*"end" + 0.161*"mu" + -0.144*"galaxy" + 0.127*"nu" + -0.127*"star"
Topic 2: 0.241*"galaxy" + -0.234*"let" + 0.214*"mass" + 0.204*"star" + 0.203*"et" + 0.189*"al" + 0.176*"hline" + 0.170*"tabular" + 0.146*"left" + 0.141*"right"
Topic 3: 0.333*"state" + -0.261*"let" + 0.244*"energy" + -0.207*"galaxy" + 0.192*"spin" + 0.155*"field" + -0.151*"star" + -0.146*"set" + -0.135*"et" + -0.131*"theorem"
Topic 4: -0.885*"ensuremath" + -0.248*"mathit" + -0.177*"mathsf" + -0.175*"bullet" + -0.174*"mathop" + -0.117*"mathord" + -0.094*"mathbb" + -0.059*"sigma" + -0.051*"frac" + -0.045*"nil"
Topic 5: 0.410*"frac" + -0.369*"hline" + -0.354*"tabular" + -0.199*"state" + -0.168*"begin" + 0.127*"right" + 0.125*"lambda" + -0.125*"cdots" + 0.124*"tau" + 0.

In [23]:
def get_lsa_topics(model, num_topics):
    word_dict = {};
    for i in range(num_topics):
        words = model.show_topic(i, topn = 20);
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in words];
    return pd.DataFrame(word_dict);

In [25]:
get_lsa_topics(lsa_model, 10)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10
0,one,right,galaxy,state,ensuremath,frac,de,width,state,right
1,model,left,let,let,mathit,hline,et,height,spin,left
2,two,frac,mass,energy,mathsf,tabular,est,eps,galaxy,field
3,function,hline,star,galaxy,bullet,state,le,includegraphics,equation,frac
4,state,tabular,et,spin,mathop,begin,que,array,site,mu
5,energy,end,al,field,mathord,right,la,end,electron,nu
6,right,mu,hline,star,mathbb,lambda,un,galaxy,lattice,mathbf
7,time,galaxy,tabular,set,sigma,cdots,pour,line,solution,time
8,case,nu,left,et,frac,tau,dans,state,hline,partial
9,field,star,right,theorem,nil,equation,si,mass,time,tau


In [29]:
# Assuming you have the LSI model trained already (lsa_model)

# Tokenize the single text instance
single_text = words(test_df['article_text'][100])
# tokenized_text = simple_preprocess(single_text)

# Create a dictionary from the tokenized text
# Note: Use the same dictionary used for training
# dictionary = Dictionary([single_text])

# Convert the tokenized text into a bag-of-words representation
bow_vector = dictionary.doc2bow(single_text)

# Use the trained LSI model for inference
topic_distribution = lsa_model[bow_vector]

# Print the topic distribution for the single test instance
print("Topic distribution:", topic_distribution)


Topic distribution: [(0, 139.29293697877313), (1, -4.2036029264488075), (2, -118.58159891988696), (3, -113.61074470787372), (4, 12.676394394153874), (5, -39.91912522025825), (6, 1.3602845240010342), (7, 1.2190027152583185), (8, 13.708182453054846), (9, 29.95134747552311)]


In [31]:
for topic_id, topic_prob in topic_distribution:
    print("Topic:", topic_id)
    print("Top Words:", lsa_model.show_topic(topic_id,topn=20))

Topic: 0
Top Words: [('one', 0.19129430526317964), ('model', 0.16538887923712728), ('two', 0.15771229061684627), ('function', 0.15235684584919632), ('state', 0.1490719528089296), ('energy', 0.14061981641607918), ('right', 0.13695452860359902), ('time', 0.13485967308850955), ('case', 0.13388361605718907), ('field', 0.13317057861157514), ('value', 0.12868845346906377), ('result', 0.1241136088842923), ('also', 0.12178006107739614), ('left', 0.1216090263600173), ('mass', 0.11036544312295955), ('system', 0.10638942308409224), ('frac', 0.10511693850811883), ('equation', 0.10330681515716575), ('point', 0.1027456594387255), ('order', 0.10115805923128575)]
Topic: 1
Top Words: [('right', 0.36949475323328146), ('left', 0.34835071302251985), ('frac', 0.33560888857878035), ('hline', 0.2525297161716446), ('tabular', 0.241883362620962), ('end', 0.19690961799629592), ('mu', 0.16054024716406104), ('galaxy', -0.1440673297195455), ('nu', 0.12726772915651183), ('star', -0.12706199906992857), ('begin', 0.1